In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import logging
import paramiko
from datetime import date

# Configure basic loggig
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Define constants and file paths
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
ENV_FILE_PATH = os.path.join(PROJECT_ROOT, '.env')
ANALYSIS_TRACKER_FILENAME = 'log_analysis_tracker.xlsx'
ANALYSIS_TRACKER_PATH = os.path.join(DATA_DIR, ANALYSIS_TRACKER_FILENAME)


# Column name in Excel containing remote log directory paths
LOG_PATH_COLUMN = 'remote_log_directory'

# Define SFTP Port for Docker container connection
SFTP_PORT = 2222

# Base dir for local logs
LOCAL_LOG_STORAGE_BASE = os.path.join(DATA_DIR, 'downloaded_logs')

# Keywords to search for in logs (Make these match your sample logs)
SUCCESS_KEYWORD = 'Execution Return Code: 0'
FAILURE_KEYWORD = '*** Failure'
ERROR_KEYWORD = '*** Error:'

# Column prefix for daily results in Excel
RESULTS_COLUMN_PREFIX = 'analysis_results_'

# --- Helper Functions ---

def get_analysis_tracker(filename: str, required_col: str) -> pd.DataFrame | None:
    """Loads the analysis tracker Excel file into a pandas DataFrame."""
    if not os.path.exists(filename):
        logging.error(f"Tracker file not found: '{filename}'. Please create it.")
        return None

    logging.info(f"Loading tracker file: '{filename}'")
    try:
        df = pd.read_excel(filename, header=0)
        logging.info(f"Successfully loaded tracker with shape: {df.shape}")
        if required_col not in df.columns:
            logging.error(f"Tracker file '{filename}' is missing the required column: '{required_col}'")
            return None
        logging.info(f"Required column '{required_col}' found in tracker.")
        return df
    except Exception as e:
        logging.error(f"Failed to read tracker file '{filename}': {e}")
        return None

def get_current_date_string() -> str:
    """Returns today's date as a string in 'YYYYMMDD' format."""
    return date.today().strftime('%Y%m%d')

def get_log_download_directory(base_dir: str, date_string: str) -> str:
    """Constructs the path to the directory holding logs for a specific date."""
    return os.path.join(base_dir, f"{date_string}_logs")

In [2]:
def update_tracker_with_results(
    df: pd.DataFrame,
    analysis_results: dict,
    date_string: str,
    problematic_remote_paths: list | None = None
) -> pd.DataFrame:
    """
    Updates the DataFrame with log analysis results for the given date.

    Args:
        df (pd.DataFrame): The DataFrame to update.
        analysis_results (dict): Dictionary mapping log filenames to statuses.
        date_string (str): The current date string (YYYYMMDD) for the results column.
        problematic_remote_paths (list | None): Optional list of remote paths
                                                 that had download/access issues.

    Returns:
        pd.DataFrame: The updated DataFrame.
    """
    results_col = f"{RESULTS_COLUMN_PREFIX}{date_string}"
    logging.info(f"Updating tracker DataFrame with results in column: {results_col}")

    # Initialize or reset the results column for the current run
    if results_col not in df.columns:
        # Insert new column before the last column if possible, otherwise append
        try:
            # Find a suitable position (e.g., after LOG_PATH_COLUMN or at the end)
            insert_pos = df.columns.get_loc(LOG_PATH_COLUMN) + 1
        except KeyError:
            insert_pos = len(df.columns) # Append if LOG_PATH_COLUMN not found
        df.insert(insert_pos, results_col, 'not_analyzed')
        logging.info(f"Added new results column: {results_col}")
    else:
        logging.info(f"Resetting existing results column: {results_col}")
        df[results_col] = 'not_analyzed' # Reset statuses for this run

    if problematic_remote_paths is None:
        problematic_remote_paths = []

    # --- Match results back to the DataFrame ---
    analyzed_rows = set() # Keep track of rows updated based on a found log file

    for log_filename, status in analysis_results.items():
        if status == 'directory_not_found': # Skip if analysis function indicated dir missing
            continue

        # --- Attempt to match log filename to DataFrame rows ---
        # Strategy: Extract job name from filename (part before '-YYYYMMDD')
        # and match it against the last component of the remote_log_directory path.
        try:
            # Handle potential variations in filename format (e.g., no date)
            base_job_name = log_filename.split('-')[0] if '-' in log_filename else os.path.splitext(log_filename)[0]
        except Exception:
            logging.warning(f"Could not extract base job name from log filename: {log_filename}")
            continue # Skip if filename format is unexpected

        matched_indices = []
        for index, row in df.iterrows():
             remote_path = str(row[LOG_PATH_COLUMN]) if pd.notna(row[LOG_PATH_COLUMN]) else None
             if remote_path:
                 # Extract expected identifier from remote path (last component)
                 path_identifier = os.path.basename(remote_path.rstrip('/'))
                 # Check if extracted base job name matches the path identifier
                 if base_job_name == path_identifier:
                     matched_indices.append(index)

        if matched_indices:
             # Update all rows that correspond to this log file/job name
             for idx in matched_indices:
                 df.loc[idx, results_col] = status
                 analyzed_rows.add(idx) # Mark this row as having its status set by an analyzed log
             logging.info(f"Matched '{log_filename}' (status: {status}) to {len(matched_indices)} tracker rows (Indices: {matched_indices}).")
        else:
             logging.warning(f"Could not match log file '{log_filename}' to any row in the tracker based on path identifier '{base_job_name}'.")

    # --- Handle rows where logs might not have been downloaded or analyzed ---
    logging.info("Performing final status updates for rows without direct log matches...")
    for index, row in df.iterrows():
        current_status = df.loc[index, results_col]
        remote_path = str(row[LOG_PATH_COLUMN]) if pd.notna(row[LOG_PATH_COLUMN]) else None

        # 1. Mark rows for paths known to have access/download errors
        if remote_path in problematic_remote_paths:
            if current_status == 'not_analyzed': # Only overwrite if not already set by analysis
                 df.loc[index, results_col] = 'access_error'
                 logging.debug(f"Row {index}: Marked as 'access_error' due to problematic path '{remote_path}'.")

        # 2. Mark rows that were not updated by any analyzed log file
        elif index not in analyzed_rows and current_status == 'not_analyzed':
            if not remote_path:
                df.loc[index, results_col] = 'missing_path'
                logging.debug(f"Row {index}: Marked as 'missing_path'.")
            else:
                # Log wasn't downloaded, wasn't found locally, or couldn't be matched
                df.loc[index, results_col] = 'log_not_found_or_analyzed'
                logging.debug(f"Row {index}: Marked as 'log_not_found_or_analyzed' for path '{remote_path}'.")

    logging.info("Tracker update process finished.")
    return df


def save_analysis_results(df: pd.DataFrame, filename: str):
    """Saves the updated DataFrame back to the Excel tracker file."""
    logging.info(f"Attempting to save updated tracker to: {filename}")
    try:
        # Use ExcelWriter for potentially better handling if needed, but direct save is fine
        df.to_excel(filename, index=False, engine='openpyxl')
        logging.info(f"Analysis results saved successfully.")
    except PermissionError:
         logging.error(f"Permission denied saving analysis results to '{filename}'. Is the file open?")
         print(f"\nERROR: Could not save tracker - Permission Denied. Close the file '{filename}' and retry.")
    except Exception as e:
        logging.error(f"Failed to save analysis results to '{filename}': {e}")
        print(f"\nERROR: Failed to save tracker file: {e}")


# --- Determine Local Log Directory ---

today_date_str = get_current_date_string()
local_log_dir_to_analyze = get_log_download_directory(LOCAL_LOG_STORAGE_BASE, today_date_str)


def analyze_downloaded_logs(local_log_dir: str) -> dict:
    """(Copied from Notebook 4 for execution context)"""
    log_analysis_results = {}
    logging.info(f"Executing analysis function for directory: {local_log_dir}")
    if not os.path.isdir(local_log_dir):
        logging.error(f"Local log directory not found: {local_log_dir}.")
        return {'error': 'directory_not_found'}
    local_log_files = os.listdir(local_log_dir)
    if not local_log_files:
        logging.warning(f"No log files found in local directory: {local_log_dir}")
        return {}
    for log_filename in local_log_files:
        local_log_path = os.path.join(local_log_dir, log_filename)
        analysis_status = 'unknown'
        try:
            with open(local_log_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.readlines()
                for line in reversed(content):
                    if SUCCESS_KEYWORD in line: analysis_status = 'success'; break
                    elif FAILURE_KEYWORD in line: analysis_status = 'failure'; break
                    elif ERROR_KEYWORD in line: analysis_status = 'error'; break
        except Exception as e:
            logging.error(f"Error parsing {log_filename}: {e}"); analysis_status = 'parse_error'
        log_analysis_results[log_filename] = analysis_status
    return log_analysis_results

analysis_results_dict = analyze_downloaded_logs(local_log_dir_to_analyze)

# Load tracker dataframe
analysis_df = get_analysis_tracker(ANALYSIS_TRACKER_PATH, LOG_PATH_COLUMN)

# Update and save
if analysis_df is not None and analysis_results_dict:
    if 'error' in analysis_results_dict and analysis_results_dict['error'] == 'directory_not_found':
         print(f"\nSkipping update: Local log directory '{local_log_dir_to_analyze}' not found.")
         logging.error(f"Local log directory '{local_log_dir_to_analyze}' not found. Cannot update tracker.")
    else:
        # Simulate problematic paths if needed for testing update logic
        problematic_paths_example = [] # Assuming none

        # Update the df
        updated_df = update_tracker_with_results(
            analysis_df.copy(), # Pass a copy to avoid modifying original in memory if needed
            analysis_results_dict,
            today_date_str,
            problematic_paths_example
        )

        # Save the updated DataFrame
        save_analysis_results(updated_df, ANALYSIS_TRACKER_PATH)

        print("\n--- Updated Tracker DataFrame Head ---")
        print(updated_df.head())
        results_col_name = f"{RESULTS_COLUMN_PREFIX}{today_date_str}"
        if results_col_name in updated_df.columns:
             print(f"\n--- Value Counts for {results_col_name} ---")
             print(updated_df[results_col_name].value_counts())

elif analysis_df is None:
     print("\nSkipping update: Failed to load the tracker DataFrame.")
     logging.error("Tracker DataFrame could not be loaded. Update skipped.")
else: # analysis_results_dict is empty or indicates error
     print(f"\nSkipping update: No analysis results generated from '{local_log_dir_to_analyze}'.")
     logging.warning(f"No analysis results available. Check directory '{local_log_dir_to_analyze}'. Update skipped.")


2025-05-01 17:01:46,922 - INFO - Executing analysis function for directory: /Users/benkaan/Desktop/projects/remote-log-analysis-automation/data/downloaded_logs/20250501_logs
2025-05-01 17:01:46,926 - INFO - Loading tracker file: '/Users/benkaan/Desktop/projects/remote-log-analysis-automation/data/log_analysis_tracker.xlsx'
2025-05-01 17:01:47,000 - INFO - Successfully loaded tracker with shape: (12, 6)
2025-05-01 17:01:47,001 - INFO - Required column 'remote_log_directory' found in tracker.
2025-05-01 17:01:47,001 - INFO - Updating tracker DataFrame with results in column: analysis_results_20250501
2025-05-01 17:01:47,002 - INFO - Added new results column: analysis_results_20250501
2025-05-01 17:01:47,002 - INFO - Matched 'job_conversion_rate-20250501_083000.log' (status: error) to 1 tracker rows (Indices: [4]).
2025-05-01 17:01:47,003 - INFO - Matched 'job_payment_proc-20250501_083000.log' (status: error) to 1 tracker rows (Indices: [9]).
2025-05-01 17:01:47,004 - INFO - Matched 'job_


--- Updated Tracker DataFrame Head ---
                            remote_log_directory analysis_results_20250501  \
0    /logs/marketing/campaigns/job_segment_users                     error   
1       /logs/marketing/campaigns/job_update_crm                   success   
2      /logs/marketing/campaigns/job_email_blast                   success   
3      /logs/marketing/analytics/job_web_traffic                   success   
4  /logs/marketing/analytics/job_conversion_rate                     error   

  analysis_results_20250430 analysis_results_20250429    project department  \
0                     error                     error  marketing  campaigns   
1                   success                   success  marketing  campaigns   
2                   success                   success  marketing  campaigns   
3                   success                   success  marketing  analytics   
4                     error                     error  marketing  analytics   

              jo